In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model = 512, h = 8):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.h = h
        self.d_k = self.d_v = self.d_model / self.h
        self.linears = nn.ModuleList([nn.Linear(self.d_model, self.d_k, bias=False)*24])
        self.linear = nn.Linear(self.h * self.d_v, self.d_model)
        self.Z = torch.randn(self.d_model, self.d_v)
        self.normalize = nn.LayerNorm(self.d_model)
        self.feed_forward = nn.Sequential(
            nn.Linear(512, 2048, bias = False),
            nn.ReLU(),
            nn.Linear(2048, 512, bias = False)
        )
    def forward(self, x):
        self.x = x
        self.heads = []
        for i, _ in enumerate(self.linears):
            Q = self.linears[3*i](self.x)
            K = self.linears[3*i+1](self.x)
            V = self.linears[3*i+2](self.x)
            head = F.softmax(torch.matmul(Q, K.T) / torch.sqrt(self.d_k)) * V
            self.heads.append(head)
        self.Z = self.linear(torch.cat((self.heads), dim=-1))
        self.Z = self.normalize(self.x + self.feed_forward(self.normalize(self.x + self.Z)))
        return self.Z